# Profitable App Profiles for the App Store and Google Play Markets

*This project is built from the perspective of being a data analyst for a company that builds free-to-install Android and iOS mobile apps in English, and consequently needs to maximize ad revenue.*

The aim of this project is to recognize the most profitable profiles for apps on the Apple App Store and Google Play markets. This is to enable developers to make data-driven decisions to decide which genre(s) of apps to build in order to maximize revenue from in-app ads.

## Exploring Data

There are two datasets that can be used for this purpose.\
[*AppleStore.csv* (link)](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) contains information on 7000+ apps found on the Apple App Store.\
[*googleplaystore.csv* (link)](https://www.kaggle.com/datasets/lava18/google-play-store-apps) contains information on 10000+ apps found on the Google Play store.\
Let's explore both of these datasets to see which of the two will be the best to use in this case.

Let's start by opening both datasets.

In [157]:
from csv import reader
opened_applestore = open('AppleStore.csv', encoding='utf8') # opening the Apple Aop Store file
read_apple = reader(opened_applestore) # using reader to read in Apple Store data
apple_data = list(read_apple) # take Apple Store data as a list of lists
apple_header = apple_data[0] #isolating the header row from the dataset
apple_data = apple_data[1:]


# repeat for the google dataset
opened_googleplay = open('googleplaystore.csv', encoding='utf8')
read_google = reader(opened_googleplay)
google_data = list(read_google)
google_header = google_data[0]
google_data = google_data[1:]

Let's write a function that dispays a digestible overview of the characteristics of each dataset.

In [158]:
def explore_data(dataset, start, end, rows_and_columns = False):
    dataset_slice =  dataset[start:end]
    for row in dataset_slice:
        print(row, '\n') # see a few rows of the dataset
    if rows_and_columns: # display the dataset's dimensions
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Now, let's take a look at the *AppleStore.csv* dataset.

In [159]:
print(apple_header, '\n')
explore_data(apple_data, 0, 5, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'] 

['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1'] 

Number of rows: 7197
Number of columns: 16


We can see that [*AppleStore.csv*](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps) has information on 7197 apps. At a glance, the columns that may be of use in this project are 'rating_count_tot', 'rating_count_ver', 'user_rating_ver', 'cont_rating', and 'prime_genre'.
Some of these column names may not be entirely self-explanatory. See the documentation to review the descriptions of each column.

And now, *googleplaystore.csv*.

In [160]:
print(google_header, '\n')
explore_data(google_data, 0, 5, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Eve

We see that [*googleplaystore.csv* (link)](https://www.kaggle.com/datasets/lava18/google-play-store-apps) has information on 10841 apps. The columns that seem potentially useful from this dataset are 'Category', 'Rating', 'Reviews', 'Installs', 'Type', and 'Genres'.
Descriptions of each column can again be found on the linked documentation page.

## Cleaning the Data

As our company in this project only builds free-to-install, English apps, data on apps that are either a) not free or b) not in English is irrelevant. So before analyzing these datasets, let's delete such rows that aren't relevant to our goal. We also need to delete erroneous data, like duplicate or inaccurate rows.

### Bad Entries

[This discussion post](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps/discussion/106176#676953) points out that index 9149 (Command and Conquer) and 10473 (Life Made Wi-Fi) have errors. The poster says that these indexes are *including* the header row. Let's check on these by comparing the headers of the dataset to these specific rows, adjusting the indexes since the header row has been removed.

In [161]:
print(google_header, '\n')
print(google_data[9148])
print(google_data[10472])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Command & Conquer: Rivals', 'FAMILY', 'NaN', '0', 'Varies with device', '0', 'NaN', '0', 'Everyone 10+', 'Strategy', 'June 28, 2018', 'Varies with device', 'Varies with device']
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


It does look like the data for the first entry the post referred to is missing a lot of data, and the second entry has a missing 'Rating' column and a column shift for the remaining columns. Lets remove these entries from the dataset.

In [162]:
del google_data[9148]
del google_data[10472]

### Duplicate Entries

Because these datasets are so large, there is a chance that they may contain duplicate entries of apps. Let's check for duplicates in the Google dataset.

In [163]:
google_uniques = [] # initializing an empty list -- this will be used to check whether each next app has already been encountered
google_duplicates = [] # will store duplicates for a bit so we can intentionally check for the most recent entry to keep and delete the rest
for row in google_data:
    name = row[0]
    if name not in google_uniques:
        google_uniques.append(name) # if the name of the current app hasn't been encountered already, add it to the list of uniques
    else: # if the name of the current app has already been encountered, the current row is a duplicate
        google_duplicates.append(row)
print(len(google_duplicates)) # prints the number of duplicate entries

1180


That's a lot of duplicates! To double check, let's pick one instance of a duplicate that was caught, and see how many instances of that specific app name there are in the entire dataset.

In [164]:
app_to_check = google_duplicates[0][0] # let's get the name of the first app in the duplicates list
for row in google_data:
    name = row[0]
    if name == app_to_check:
        print(row)

['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80804', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']


Duplicates, indeed. Now, if we delete the duplicates randomly (for example, by deleting the first appearance of the app name in the dataset until only one instance remains), we risk compromising the accuracy of the remaining data by accidentally deleting the more recent (e.g. the most accurate) entry.\
Q: How do we check to see which entry of a series of duplicates is the most recent?\
A: Look at the numbers! If a series of duplicates of the same app have differing numbers for 'Reviews', the entry with the highest number is likely the most recent. So we should preserve that entry and get rid of the rest.